<a href="https://colab.research.google.com/github/TeenPhilosopher/audiocraft/blob/main/BetterMusicGenDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tempfile import NamedTemporaryFile
import argparse
import torch
import gradio as gr
import os
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
def predict(model, text, melody, window_len_secs, total_duration_secs, slide_secs, topk, topp, temperature, cfg_coef):
    global MODEL
    topk = int(topk)
    if MODEL is None or MODEL.name != model:
        MODEL = load_model(model)
    print(MODEL.lm.fuser)
    print(MODEL.lm.fuser.fuse2cond)
    MODEL.set_generation_params(use_sampling=True, top_k=topk,
                              top_p=topp, temperature=temperature,
                              duration=window_len_secs, cfg_coef=cfg_coef)
    print(MODEL.generation_params['cfg_coef'])
    if melody:
        sr, melody = melody[0], torch.from_numpy(melody[1]).to(MODEL.device).float().t().unsqueeze(0)
        print(melody.shape)
        if melody.dim() == 2:
            melody = melody[None]
        melody = melody[..., :int(sr * total_duration_secs)]
    else:
        sr, melody = None, None
    output = MODEL.generate_music_for_duration(description=text, melody=melody, melody_sr=sr, window_len_secs=window_len_secs, total_duration_secs=total_duration_secs, slide_secs=slide_secs)
    output = output.detach().cpu().float()[0]
    with NamedTemporaryFile("wb", suffix=".wav", delete=False) as file:
        audio_write(
            file.name, output, MODEL.sample_rate, strategy="loudness",
            loudness_headroom_db=16, loudness_compressor=True, add_suffix=False)
        waveform_video = gr.make_waveform(file.name)
    return waveform_video

In [ ]:
!python3 -m pip install -U git+https://github.com/TeenPhilosopher/audiocraft#egg=audiocraft
# !python3 -m pip install -U audiocraft

In [ ]:
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch

In [ ]:
model = musicgen.MusicGen.get_pretrained('medium', device='cuda')
descriptions = [
    "Strong repeating lofi beats, warm mellow synths and warm jazzy chords with repeating vinyl crackle and repeating ocean wave and rain sounds, background conversation, slow subtle melody",
    "Strong repeating EDM beat, a light and cheery EDM track, with syncopated drums, warm harmony airy pads, and strong emotions",
    "Raw indie rock vibe with strong guitar riffs, powerful drum beats, bass guitar driving the rhythm, and a melodic vocal line layered over the top",
    "Ambient music that is quiet, slowly evolving, with minimal beats, long echoing drones, and occasional soft piano and strings",
    "Epic orchestral music with strong brass melodies, powerful percussions, dramatic string sections and a sense of adventure",
    "Jazz music featuring swing rhythms, improvisation, syncopation, walking double bass and iconic brass solos",
    "New age synth music with ethereal pads, subtle rhythms, ambient textures, and meditative melodies"
]
for description in descriptions:
    res = model.generate_music_for_duration(description=description,melody=None,melody_sr=None,window_len_secs=30,total_duration_secs=70,slide_secs=20)
    display_audio(res, 32000)